to do
loop through all states and append to one file
add state to appended file as you go - append with or without state underscore?


In [17]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [18]:
# URL of page to be scraped
# NEED TO ADD LOOP TO THIS FOR ALL STATES
url = 'https://en.wikipedia.org/wiki/List_of_radio_stations_in_New_York'

In [19]:
# NEED TO ADD LOOP TO THIS FOR ALL STATES
#Read In HTML
tables = pd.read_html(url)
#Create a Dateframe From First Table
table_df = pd.DataFrame(tables[0])
#Set Column Names From Index 0
table_df.columns = table_df.iloc[0]
#Drop Duplicate Row[0]
table_df = table_df.reindex(table_df.index.drop(0))
#Rename Weird Columns
table_df = table_df.rename(columns={"City of License [1][2]":"Location","Format [3]":"Format"})
#Sample Date
table_df.head()

0 Call sign Frequency          Location  \
1      WAAL   99.1 FM        Binghamton   
2      WABC    770 AM     New York City   
3      WABH   1380 AM              Bath   
4      WABY    900 AM  Saratoga Springs   
5      WACK   1420 AM            Newark   

0                                  Licensee                 Format  
1  Townsquare Media Binghamton License, LLC           Classic rock  
2                Radio License Holdings LLC              News/Talk  
3                   Tower Broadcasting, LLC                 Sports  
4           Empire Broadcasting Corporation                 Silent  
5                        Waynco Radio, Inc.  News/Talk Information

In [20]:
county_url = "https://en.wikipedia.org/wiki/List_of_towns_in_New_York_(state)"
cities_url = "https://en.wikipedia.org/wiki/List_of_cities_in_New_York_(state)"

In [21]:
county_table = pd.read_html(county_url)
cities_table = pd.read_html(cities_url)

In [22]:
county_df = pd.DataFrame(county_table[1])
cities_df = pd.DataFrame(cities_table[1])

In [23]:
#Set Column Names From Index 0
county_df.columns = county_df.iloc[0]
cities_df.columns = cities_df.iloc[0]
#Drop Duplicate Row[0]
county_df = county_df.reindex(county_df.index.drop(0))
cities_df = cities_df.reindex(cities_df.index.drop(0))


In [24]:
#Select Useful Columns
county_df = county_df[['Town','County']]
cities_df = cities_df[['City','County']]
#rename columns
county_df = county_df.rename(columns={"Town":"Location"})
cities_df = cities_df.rename(columns={"City":"Location"})


In [26]:
#append cities to towns
final_counties_df = county_df.append(cities_df)

#data cleaning "New York" to "New York City"
final_counties_df["Location"] = final_counties_df["Location"].replace(
   {"New York":"New York City"})

#Merge two data-frames
final_scrape_df = pd.merge(final_counties_df, table_df, on='Location')

#import radio csv as a dataframe
radio_df = pd.read_csv("formats.csv")

#strip out extra spaces in the format values in final_scrape_df
final_scrape_df.columns = final_scrape_df.columns.str.replace(' ','')


#join radio dataframe on "Format"
final_scrape_df = pd.merge(final_scrape_df,radio_df, on="Format")

#drop "Format"
final_scrape_df= final_scrape_df.drop(['Format'], axis=1)

#change Revised Format to Format
final_scrape_df = final_scrape_df.rename(columns={"Revised Format":"Format"})

final_scrape_df


Location                                        County Callsign  \
0           Alfred                                      Allegany     WALF   
1           Alfred                                      Allegany     WETD   
2         Angelica                                      Allegany  WRAQ-LP   
3       Binghamton                                        Broome  WBDY-LP   
4       Binghamton                                        Broome  WBDY-LP   
5        Cazenovia                                       Madison     WITC   
6          Clinton                                       Clinton  WHCL-FM   
7          Clinton                                      Dutchess  WHCL-FM   
8          Corning                                       Steuben     WCEB   
9          Corning                                       Steuben     WCEB   
10          Fenner                                       Madison     WMVQ   
11       Hempstead                                        Nassau     WRHU   
12        Herkimer                                      Herkimer     WVHC   
13          Mohawk                                    Montgomery  WHMV-LP   
14        Ossining                                   Westchester     WOSS   
15         Pawling                                      Dutchess  WPWL-LP   
16        Pulteney                                       Steuben  WQKA-LP   
17       Rochester                                        Ulster  WAYO-LP   
18       Rochester                                        Ulster  WRFZ-LP   
19       Rochester                                        Ulster  WXIR-LP   
20       Rochester                                        Monroe  WAYO-LP   
21       Rochester                                        Monroe  WRFZ-LP   
22       Rochester                                        Monroe  WXIR-LP   
23         Roxbury                                      Delaware     WIOX   
24          Albany                                        Albany  WCAA-LP   
25          Albany                                        Albany  WOOA-LP   
26        Cortland                                      Cortland  WSUC-FM   
27    Gloversville                                        Fulton     WFNY   
28     Schenectady                                   Schenectady  WOOS-LP   
29            Troy                                    Rensselaer  WOOC-LP   
..             ...                                           ...      ...   
540       Westport                                         Essex     WCLX   
541      Willsboro                                         Essex     WXZO   
542      Woodstock                                        Ulster  WIOF-LP   
543      Jamestown                                    Chautauqua  WRFA-LP   
544         Albany                                        Albany     WCDB   
545         Albany                                        Albany   WGY-FM   
546    Schenectady                                   Schenectady      WGY   
547        Buffalo                                          Erie     WBBF   
548        Buffalo                                          Erie     WBEN   
549        Buffalo                                          Erie     WBNY   
550        Buffalo                                          Erie  WNED-FM   
551      Jamestown                                    Chautauqua     WNJA   
552        Buffalo                                          Erie     WWKB   
553        Buffalo                                          Erie     WWWS   
554       Cortland                                      Cortland     WYBY   
555    Glens Falls                                        Warren     WGFR   
556   Gloversville                                        Fulton     WENT   
557        Hornell                                       Steuben     WLEA   
558      Jamestown                                    Chautauqua     WJTN   
559   New Rochelle                                   Westchester     WVIP   
560   

In [27]:
#Ping the endpoint thing
population_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H010001&for=county:*&in=state:36&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"


In [28]:
# Request data
population = requests.get(population_url).json()

#Create a dataframe
population_df = pd.DataFrame(population)

#create column headers
population_df.columns = population_df.iloc[0]
#remove old row[0]
population_df = population_df.reindex(population_df.index.drop(0))
#Split County, State Column
population_df["NAME"] = population_df["NAME"].str.split(" ", n = 1, expand = True)
#Rename Column as County
population_df = population_df.rename(columns={"NAME":"County","H010001":"Population"})
#Drop Unnecessary Columns
population_df = population_df[['County','Population']]


In [29]:
#Merge county graph with population with final_scrape which has radio data 
pop_radio_df = pd.merge(population_df, final_scrape_df, on ='County')

#Clean up formats
pop_radio_df["Format"] = pop_radio_df["Format"].replace(
    {'Adult album alternative/Talk':'Adult album alternative', 
     'Adult contemporary & News': 'Adult contemporary',
     'Alternative': 'Alternative radio',
     'Catholic Talk/EWTN' :'Catholic',
     'Christian (Family Life Network)':'Christian',
     'Christian Contemporary (Air1)': 'Christian',
     'Christian Contemporary (K-Love)': 'Christian',
     'Ethnic/Greek':'Ethnic',
    })

In [30]:
#Determine # of formats per county
format_ser= pop_radio_df.groupby(['County', 'Format']).Format.count()
format_df=format_ser.unstack()
format_df_clean= format_df.fillna(0)


In [31]:
#Add population
final_df = pd.merge(population_df, format_df_clean, on ='County')
final_df.head()

County Population  Alternative radio  Classical  Contemporary  Country  \
0     Genesee      58171                0.0        0.0           2.0      0.0   
1      Greene      45821                0.0        0.0           2.0      0.0   
2  Livingston      59635                0.0        0.0           0.0      0.0   
3    Herkimer      63093                0.0        0.0           0.0      0.0   
4    Hamilton       4756                0.0        0.0           0.0      0.0   

   Ethnic  Jazz  Oldies  Other  Religious  Rock  Spanish  Sports  Standards  \
0     0.0   0.0     0.0    0.0        0.0   2.0      0.0     0.0        0.0   
1     0.0   0.0     0.0    0.0        0.0   0.0      0.0     0.0        0.0   
2     0.0   0.0     0.0    0.0        0.0   0.0      0.0     0.0        0.0   
3     0.0   0.0     1.0    0.0        1.0   2.0      0.0     2.0        0.0   
4     0.0   0.0     0.0    0.0        0.0   0.0      0.0     0.0        0.0   

   Talk  Top40  Urban  Variety  
0   0.0    0.0    0.0      0.0  
1   0.0    0.0    0.0      0.0  
2   1.0    0.0    0.0      0.0  
3   0.0    2.0    0.0      1.0  
4   1.0    0.0    0.0      0.0

In [32]:
final_df.to_excel("output.xlsx")

In [16]:
#cleanup since I'm getting duplicate column name error in sqlite
headers = list(final_df.columns.values)
headers

['County',
 'Population',
 '1950s Oldies',
 '1960s and 1970s Oldies',
 'AAA, progressive, blues, R&B, jazz & folk',
 'AAA/NPR News',
 'AC/Talk',
 'Active rock',
 'Adult Standards',
 'Adult album alternative',
 'Adult contemporary',
 'Adult hits',
 'Adult standards',
 'Album Adult Alternative',
 'Album Rock',
 'Album-oriented rock',
 'Alternative radio',
 'Alternative rock',
 'Americana',
 'Bollywood (Indian) Music',
 'Campus radio',
 'Catholic',
 'Christian',
 'Classic country',
 'Classic hip-hop',
 'Classic hits',
 'Classic rock',
 'Classical',
 'Classical music',
 'College',
 'Contemporary Christian',
 'Country',
 'Country music',
 'Ethnic',
 'Freeform',
 'Freeform/Alternative',
 'Full service',
 'Full service/news/talk',
 'Full-service',
 'Full-service/classic hits',
 'Hip-hop',
 'Hispanic',
 'Hot AC',
 'Hot adult contemporary',
 'Independent',
 'Jazz',
 'Jazz, Ethnic',
 'Jazz/Community',
 'Jazz/mixed',
 'Mainstream jazz',
 'Mainstream rock',
 'Modern rock',
 'NPR',
 'News radio',
 